In [3]:
import pandas as pd
import os
from pyproj import Transformer

# 프로젝트 최상위 폴더 절대경로 (본인 환경에 맞게 수정)
BASE_DIR = r"C:\Users\Admin\Desktop\pick-your-place"
os.chdir(BASE_DIR)
print("현재 작업 디렉터리:", os.getcwd())

from geocoding.vworld_geocode import road_address_to_coordinates, coordinates_to_jibun_address, road_to_jibun_address
from geocoding.admin_mapper import extract_gu_and_dong, get_gu_dong_codes

현재 작업 디렉터리: C:\Users\Admin\Desktop\pick-your-place


In [4]:
def load_store_csv(path="data/raw/store__raw.csv"):
    if not os.path.exists(path):
        raise FileNotFoundError(f"파일이 존재하지 않습니다: {path}")
    df = pd.read_csv(path, encoding="utf-8-sig")
    print(f"[로드 완료] {path} - {len(df)}건")
    return df

In [14]:
def process_store_data(df):
    # '영업중' 데이터 필터링
    df = df[df['DTLSTATENM'] == '정상영업']

    # 필요한 컬럼 리스트
    needed_cols = ['MGTNO', 'SITEWHLADDR', 'RDNWHLADDR', 'BPLCNM', 'X', 'Y']

    # 존재하는 컬럼만 선택
    selected_cols = [col for col in needed_cols if col in df.columns]
    df = df[selected_cols]

    # 컬럼명 영문으로 변경
    rename_map = {
        'MGTNO': 'store_code',
        'SITEWHLADDR': 'jibun_address',
        'RDNWHLADDR': 'road_address',
        'BPLCNM': 'store_name',
        'X': 'longitude',
        'Y': 'latitude'
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
    print(f"[전처리 완료] {len(df)}건")
    return df


In [6]:
def tm_to_lonlat(x, y):
    try:
        transformer = Transformer.from_crs("epsg:5181", "epsg:4326", always_xy=True)
        lon, lat = transformer.transform(x, y)
        return lon, lat
    except Exception as e:
        print(f"[좌표 변환 실패] X={x}, Y={y}, 에러: {e}")
        return None, None

def convert_coords(df, x_col='longitude', y_col='latitude'):
    def safe_convert(row):
        try:
            return tm_to_lonlat(row[x_col], row[y_col])
        except Exception as e:
            print(f"[⚠️ 변환 실패] X: {row.get(x_col)}, Y: {row.get(y_col)} → {e}")
            return (None, None)

    coords = df.apply(safe_convert, axis=1)
    df['lon'], df['lat'] = zip(*coords)
    return df


In [7]:
def is_within_seoul(lon, lat):
    if lon is None or lat is None:
        return False
    return 126.76 <= lon <= 127.19 and 37.41 <= lat <= 37.70

In [8]:
def safe_jibun_address(row):
    # 1. 기존 지번주소 있으면 우선 사용
    raw_jibun = row.get('jibun_address', '')
    if isinstance(raw_jibun, str) and raw_jibun.strip():
        return raw_jibun.strip()

    # 2. 도로명주소 → 좌표 → 지번주소
    try:
        if isinstance(row.get('road_address', ''), str) and row['road_address'].strip():
            lon, lat = road_address_to_coordinates(row['road_address'])
            if is_within_seoul(lon, lat):
                addr = coordinates_to_jibun_address(lon, lat)
                if addr and '검색결과가 없습니다' not in addr:
                    return addr
    except Exception as e:
        print(f"[도로명주소→지번주소 실패] {row.get('road_address')} → {e}")

    # 3. TM좌표 → 위경도 → 지번주소
    try:
        if is_within_seoul(row.get('lon'), row.get('lat')):
            addr = coordinates_to_jibun_address(row['lon'], row['lat'])
            if addr and '검색결과가 없습니다' not in addr:
                return addr
    except Exception as e:
        print(f"[TM좌표→지번주소 실패] ({row.get('lon')}, {row.get('lat')}) → {e}")

    # 4. 도로명주소 → 지번주소 (백업용)
    try:
        addr = road_to_jibun_address(row.get('road_address', ''))
        if addr and addr.strip():
            return addr
    except Exception as e:
        print(f"[도로명주소 백업 실패] {row.get('road_address')} → {e}")

    print(f"[⚠️ 지번주소 실패] 스토어: {row.get('store_name')}, 도로명: {row.get('road_address', '')}")
    return None


In [9]:
def safe_extract_gu_dong(addr):
    try:
        if addr:
            return pd.Series(extract_gu_and_dong(addr))
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"[주소 파싱 실패] {addr} → {e}")
        return pd.Series([None, None])

In [10]:
def safe_get_codes(row):
    try:
        return pd.Series(get_gu_dong_codes(row['gu_name_from_jibun'], row['dong_name_from_jibun']))
    except Exception as e:
        print(f"[법정→행정 매핑 실패] gu={row.get('gu_name_from_jibun')}, dong={row.get('dong_name_from_jibun')}")
        return pd.Series([None, None])

In [11]:
def mapping_process(df):
    df = process_store_data(df)

    # TM 좌표를 위경도로 변환
    df = convert_coords(df, x_col='longitude', y_col='latitude')

    # 지번주소 보완
    df['jibun_address'] = df.apply(safe_jibun_address, axis=1)

    # 자치구, 행정동명 추출
    df[['gu_name_from_jibun', 'dong_name_from_jibun']] = df['jibun_address'].apply(safe_extract_gu_dong)

    # 코드 매핑
    df[['gu_code', 'dong_code']] = df.apply(safe_get_codes, axis=1)

    return df

In [15]:
df_raw = load_store_csv()

print("원본 데이터 컬럼:", df_raw.columns.tolist())
print("DTLSTATENM unique values:", df_raw['DTLSTATENM'].unique())
print("DTLSTATENM == '영업중' 건수:", (df_raw['DTLSTATENM'] == '영업중').sum())

df_filtered = df_raw[df_raw['DTLSTATENM'] == '영업중']
print("필터링 후 컬럼 존재 여부:")
print("X 컬럼 있음?", 'X' in df_filtered.columns)
print("Y 컬럼 있음?", 'Y' in df_filtered.columns)
print("필터링 후 행 수:", len(df_filtered))


[로드 완료] data/raw/store__raw.csv - 1019건
원본 데이터 컬럼: ['OPNSFTEAMCODE', 'MGTNO', 'APVPERMYMD', 'APVCANCELYMD', 'TRDSTATEGBN', 'TRDSTATENM', 'DTLSTATEGBN', 'DTLSTATENM', 'DCBYMD', 'CLGSTDT', 'CLGENDDT', 'ROPNYMD', 'SITETEL', 'SITEAREA', 'SITEPOSTNO', 'SITEWHLADDR', 'RDNWHLADDR', 'RDNPOSTNO', 'BPLCNM', 'LASTMODTS', 'UPDATEGBN', 'UPDATEDT', 'UPTAENM', 'X', 'Y', 'JPSENM']
DTLSTATENM unique values: ['폐업처리' '직권취소' '정상영업' '영업개시전' nan '휴업처리']
DTLSTATENM == '영업중' 건수: 0
필터링 후 컬럼 존재 여부:
X 컬럼 있음? True
Y 컬럼 있음? True
필터링 후 행 수: 0


In [16]:
df_raw = load_store_csv()
df_processed = mapping_process(df_raw)

# 저장 경로
output_path = os.path.abspath("data/processed/store__processed.csv")
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_processed.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"저장 완료: {output_path}")

# 매핑 실패 데이터 따로 저장 (dong_code가 없는 경우)
if 'dong_code' in df_processed.columns:
    failures = df_processed[df_processed['dong_code'].isnull()]
    failures_path = os.path.abspath("data/processed/store__failed.csv")
    failures.to_csv(failures_path, index=False, encoding='utf-8-sig')
    print(f"⚠️ 매핑 실패 데이터 저장 완료: {failures_path}")
else:
    print("[WARNING] 'dong_code' 컬럼이 없어 매핑 실패 데이터 저장하지 않음.")

[로드 완료] data/raw/store__raw.csv - 1019건
[전처리 완료] 704건
응답 내용: {"BLD_NM" : "리첸시아 용산B", "EPSG_4326_X" : "126.967987947", "EPSG_4326_Y" : "37.537981164", "JUSO" : "서울특별시 용산구 백범로 341 (원효로1가,리첸시아 용산B)"}
지번 응답: {"ADDR" : "서울특별시 용산구 문배동 40-31"}
응답 내용: {"result":"검색결과가 없습니다."}
[주소→좌표 실패] 서울특별시 송파구 동남로 189 (가락동, 쌍용아파트)
지번 응답: {"ADDR" : "서울특별시 송파구 가락동 136-5"}
응답 내용: {"BLD_NM" : "청담 신원아침도시마인", "EPSG_4326_X" : "127.049680053", "EPSG_4326_Y" : "37.526440427", "JUSO" : "서울특별시 강남구 도산대로85길 30 (청담동,청담 신원아침도시마인)"}
지번 응답: {"ADDR" : "서울특별시 강남구 청담동 122-35"}
응답 내용: {"BLD_NM" : "논현두산위브아파트(2단지)", "EPSG_4326_X" : "127.038308184", "EPSG_4326_Y" : "37.512286346", "JUSO" : "서울특별시 강남구 언주로122길 25 (논현동,논현두산위브아파트(2단지))"}
지번 응답: {"ADDR" : "서울특별시 강남구 논현동 258"}
응답 내용: {"BLD_NM" : "브라운스톤 레전드", "EPSG_4326_X" : "127.042313197", "EPSG_4326_Y" : "37.515494243", "JUSO" : "서울특별시 강남구 선릉로 660 (삼성동,브라운스톤 레전드)"}
지번 응답: {"ADDR" : "서울특별시 강남구 삼성동 8-2"}
응답 내용: {"result":"검색결과가 없습니다."}
[주소→좌표 실패] 서울특별시 송파구 올림픽로 435 (신천동, 파크리오아파트)
지번 응답: